In [ ]:
%load_ext dotenv
%dotenv

import os

%cd {os.getenv("PROJECT_ROOT") or ".."}

%load_ext autoreload
%autoreload 1

from IPython.display import display


In [ ]:
import logging

import pandas as pd

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
import tensorflow as tf

tf.config.set_visible_devices([], "GPU")

In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(
    nb_workers=os.cpu_count(),
    progress_bar=True,
    verbose=0
)


In [ ]:
def show_df(df: pd.DataFrame):
    display(df.head())
    print(df.shape)


In [ ]:
import datasets

ds = datasets.load_from_disk("data/processed/owt_gpt2_bs1024")
logger.info(f"Loaded dataset with {len(ds)} records.")

ds


In [ ]:
import evagpt.data.utils

train_loader, _ = evagpt.data.utils.get_dataloader(
    dataset=ds["train"],
    batch_size=64,
    shuffle=True,
)
train_loader

In [ ]:
for batch in train_loader:
    print(batch["input_ids"].shape)
    break

In [ ]:
%aimport evagpt.gpt2
import jax.random as jrandom

import evagpt.gpt2

rng = jrandom.PRNGKey(0)
config = evagpt.gpt2.GPT2Config()

rng, key_model = jrandom.split(rng)
model = evagpt.gpt2.GPT2(config, key=key_model)

model

In [ ]:
import equinox as eqx
import jax

devices = jax.devices()

replicated_model = eqx.filter_jit(
    lambda m: jax.device_put_replicated(m, devices)
)(model)
